# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по одному баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 02.10.24 23:59__

__Жесткий дедлайн: 05.10.24 23:59__


### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в телефонах, поисковой строке или приложении почты. Полученную систему вам нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex), чтобы ей можно было удобно пользоваться, а так же, чтобы убедиться, что все работает как надо. В этот раз вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей предположительно одинаковых по сложности. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы получилось в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание
При сдаче зададания в anytask вам будет необходимо сдать весь код, а также отчет с подробным описанием техник, которые в применили для создания вашей системы. Не лишним будет также написать и о том, что у вас не получилось и почему.

За часть с заданиями можно будет получить до __5__ баллов, за отчет – до __3__ баллов и еще __2__ балла можно будет получить за демонстрацию вашей системы и пользовательского интерфейса. Демонстрацию прикрепляйте в anytask в виде 1-2 минутной записи экрана.

## Часть 1

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd

emails = pd.read_csv('emails.csv')
len(emails)

517401

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (1 балл).__ Очистите корпус текстов по вашему усмотрению. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст. Оценка будет выставляться по близости вашего результата к этому идеалу.

In [2]:
import re
import email
from email import policy

def get_email_body_from_forward_part(email_content):
    headers_pattern = re.compile(r"Subject:\s*(.*?)\r?\n")

    match = headers_pattern.search(email_content)

    if not match:
        return email_content

    body_start = match.end()

    return email_content[body_start:].strip()


def extract_email_body(email_text):
    msg = email.message_from_string(email_text, policy=policy.default)
    
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                encoding = part.get_content_charset() or "utf-8"

                return part.get_payload(decode=True).decode(encoding).lower()
    encoding = msg.get_content_charset() or "utf-8"

    text = msg.get_payload(decode=True).decode(encoding)

    several_emails = None
    if "----- Forwarded by" in text:
        several_emails = "----- Forwarded by"
    elif "-----Original Message" in text:
        several_emails = "-----Original Message"
    if several_emails:
        forwarded_bodies = []
        forwarded_parts = text.split(several_emails)

        for part in forwarded_parts[1:]:
            forwarded_bodies.append(get_email_body_from_forward_part(part))
        
        return re.sub(r'[^\S ]+', ' ', " ".join(forwarded_bodies).lower())
    
    return re.sub(r'[^\S ]+', ' ', text.lower())

emails['cleaned_message'] = emails['message'].apply(extract_email_body)

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы считаете, что результат получается лучше без нее, то можете удалить все небуквенные символы на этапе токенизации.

In [3]:
import re

from collections import Counter
from tqdm import tqdm

class WhitespaceOutput:
    def __init__(self, tokens):
        self.tokens = tokens
        self.ids = None


class WhitespaceTokenizer:
    def __init__(self, corpus, vocab_size=-1, n=2):
        self.n = n
        self.regex = re.compile(r'\w+|[^\w\s]+')

        words = []
        for line in tqdm(corpus):
            words.extend(self.regex.findall(line))
        
        word_count = Counter(words)
        if vocab_size == -1:
            self.vocab = set(word_count)
        else:
            common = word_count.most_common(vocab_size - 3)
            self.vocab, _ = zip(*common)
            self.vocab = set(self.vocab)

        self.vocab |= set(['[UNK]', '[BOS]', '[EOS]'])
    
    def encode(self, text):
        words = self.regex.findall(text)
        encoded = [w.lower() if w.lower() in self.vocab else '[UNK]' for w in words]

        encoded = ['[BOS]'] * self.n + encoded + ['[EOS]']
        return WhitespaceOutput(encoded)

    def get_vocab(self):
        return self.vocab
    

def get_tokenizer(corpus, vocab_size=32768, n=2):
    return WhitespaceTokenizer(corpus, vocab_size, n=n)

## Дополнение слова

Описанная система будет состоять из двух частей: дополнение слова до целого и генерация продолжения текста (или вариантов продолжений). Начнем с первой части.

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова за $O(n + m)$, где $m$ – число подходящих слов.

__Задание 2 (1 балл).__ Допишите префиксное дерево для поиска слов по префиксу. Ваше дерево должно работать за $O(n + m)$ операции, в противном случае вы не получите баллов за это задание.

In [4]:
from typing import List

class PrefixTreeNode:
    def __init__(self):
        # словарь с буквами, которые могут идти после данной вершины
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()
        
        for word in vocabulary:
            self._insert_word(word)
    
    def _insert_word(self, word: str):
        current_node = self.root
        for letter in word:
            if letter not in current_node.children:
                current_node.children[letter] = PrefixTreeNode()
            current_node = current_node.children[letter]

        current_node.is_end_of_word = True

    def search_prefix(self, prefix) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """

        current_node = self.root

        for letter in prefix:
            if letter not in current_node.children:
                return []
            current_node = current_node.children[letter]

        return self._dfs(current_node, prefix)

    def _dfs(self, node: PrefixTreeNode, prefix: str) -> List[str]:
        """
        Выполняет поиск всех слов, начиная с текущего узла в дереве.
        """
        words = []

        if node.is_end_of_word:
            words.append(prefix)
        
        for letter, next_node in node.children.items():
            words.extend(self._dfs(next_node, prefix + letter))
        
        return words

In [5]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его встречаемости в корпусе.

__Задание 3 (1 балл).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [6]:
from itertools import chain

class WordCompletor:
    def __init__(self, corpus):
        """
        corpus: list – корпус текстов
        """
        self.words_count = Counter(list(chain.from_iterable(corpus)))
        self.prefix_tree = PrefixTree(list(self.words_count.keys()))

    def get_words_and_probs(self, prefix: str) -> (List[str], List[float]):
        """
        Возвращает список слов, начинающихся на prefix,
        с их вероятностями (нормировать ничего не нужно)
        """
        words, probs = [], []
        
        words = self.prefix_tree.search_prefix(prefix.lower())

        probs = [self.words_count[word] / sum(self.words_count.values()) for word in words]

        return words, probs

In [7]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему несколько следующих слов с учетом дописанного. Для этого мы воспользуемся n-граммами и будем советовать n следующих слов. Но сперва нужно получить n-граммную модель.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

Тогда, нам нужно оценить $P(w_i \mid w_{i-1}, \dots, w_{i-n})$ по частоте встречаемости n-граммы.   

__Задание 4 (1 балл).__ Напишите класс для n-граммной модели. Понятное дело, никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [8]:
from collections import defaultdict, Counter
from tqdm import tqdm
import numpy as np

class NGramLanguageModel:
    def __init__(self, corpus, n, tokenizer: WhitespaceTokenizer | None = None):
        self.n = n

        self.counts = self._count_ngrams(corpus)

        self.prefix_counts = dict()
        for prefix, token_count in self.counts.items():
            self.prefix_counts[prefix] = sum(token_count.values())

    def _count_ngrams(self, corpus: list[list[str]] | list[str]) -> dict[str, Counter]:
        counts = defaultdict(Counter)

        for text in tqdm(corpus, desc="Counting Ngrams"):
            for i in range(self.n, len(text)):
                prefix = tuple(text[i - self.n:i])
                token = text[i]
                counts[prefix][token] += 1
            
        return counts

    def get_next_words_and_probs(self, prefix: list[str] | str) -> tuple[List[str], List[float]]:
        """
        Возвращает список слов, которые могут идти после prefix,
        а так же список вероятностей этих слов
        """

        next_words, probs = [], []
        
        prefix = self._get_last_n_words(prefix)

        possible_tokens = self.counts[prefix]

        if len(possible_tokens) == 0:
            return ['[EOS]'], [1.0]

        next_words = list(possible_tokens.keys())

        counts = list(possible_tokens.values())

        probs = np.array(counts) / self.prefix_counts[prefix]

        return next_words, probs
    
    def _get_last_n_words(self, prefix: list[str] | str) -> tuple[str]:
        if isinstance(prefix, str):
            prefix = self.tokenizer.encode(prefix).tokens[:-1]
        
        return tuple(prefix[-self.n:])

In [9]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Counting Ngrams: 100%|██████████| 3/3 [00:00<00:00, 25890.77it/s]


Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (1 балл).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. Если вы справитесь, то сможете можете добавить опцию поддержки нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [10]:
class TextSuggestion:
    def __init__(self, word_completor: WordCompletor, n_gram_model: NGramLanguageModel):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: str | list, n_words=3, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """

        suggestions = []

        if isinstance(text, str):
            text = text.split()
        
        for _ in range(n_texts):
            completions, completions_probs = self.word_completor.get_words_and_probs(text[-1])
            most_probable_completion = completions[max(enumerate(completions_probs), key=lambda x: x[1])[0]]
            text_copy = text.copy()
            suggestion = [most_probable_completion]
            for _ in range(n_words):
                next_words, next_words_probs = self.n_gram_model.get_next_words_and_probs(text_copy)
                most_probable_next_word = next_words[max(enumerate(next_words_probs), key=lambda x: x[1])[0]]
                suggestion.append(most_probable_next_word)
                text_copy.append(most_probable_next_word)

            suggestions.append(suggestion)

        return suggestions

In [11]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

Counting Ngrams: 100%|██████████| 3/3 [00:00<00:00, 78643.20it/s]


In [12]:
all_cleaned_emails = emails['cleaned_message'].values
tokenizer = get_tokenizer(all_cleaned_emails, 65536)

100%|██████████| 517401/517401 [00:20<00:00, 25141.71it/s]


In [13]:
emails_corpus = [tokenizer.encode(clean_email).tokens for clean_email in tqdm(emails['cleaned_message'])]

100%|██████████| 517401/517401 [00:40<00:00, 12810.45it/s]


In [14]:
word_completor_emails = WordCompletor(emails_corpus)
n_gram_model_email = NGramLanguageModel(corpus=emails_corpus, n=2)
text_suggestion = TextSuggestion(word_completor_emails, n_gram_model_email)

Counting Ngrams: 100%|██████████| 517401/517401 [01:54<00:00, 4524.40it/s] 


In [15]:
import dill
import sys

sys.setrecursionlimit(100000)

with open('tokenizer.pkl', 'wb') as file:
    dill.dump(tokenizer, file, byref=True)

with open('text_suggestion_em.pkl', 'wb') as file:
    dill.dump(text_suggestion, file, byref=True)

/Users/andrewut/Documents/nlp/hw1/nlp_venv/lib/python3.11/site-packages/dill/_dill.py:422: PicklingWarning: Cannot locate reference to <class '__main__.WhitespaceTokenizer'>.
  StockPickler.save(self, obj, save_persistent_id)
/Users/andrewut/Documents/nlp/hw1/nlp_venv/lib/python3.11/site-packages/dill/_dill.py:422: PicklingWarning: Cannot locate reference to <class '__main__.TextSuggestion'>.
  StockPickler.save(self, obj, save_persistent_id)
/Users/andrewut/Documents/nlp/hw1/nlp_venv/lib/python3.11/site-packages/dill/_dill.py:422: PicklingWarning: Cannot locate reference to <class '__main__.WordCompletor'>.
  StockPickler.save(self, obj, save_persistent_id)
/Users/andrewut/Documents/nlp/hw1/nlp_venv/lib/python3.11/site-packages/dill/_dill.py:422: PicklingWarning: Cannot locate reference to <class '__main__.PrefixTree'>.
  StockPickler.save(self, obj, save_persistent_id)
/Users/andrewut/Documents/nlp/hw1/nlp_venv/lib/python3.11/site-packages/dill/_dill.py:422: PicklingWarning: Cannot l

## Часть 2

Настало время довести вашу систему до ума. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным, __кроме использования дополнительных данных__. Главное – вы должны обернуть вашу систему в пользовательский интерфейс с помощью [reflex](https://github.com/reflex-dev/reflex). В нем можно реализовать почти любой функционал по вашему желанию.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

При сдаче решения прикрепите весь ваш __код__, __отчет__ по второй части и __видео__ с демонстрацией работы вашей системы. Удачи!